In [ ]:
!./k8s/install.sh

In [1]:
!task helm:build CHART=apache-jena

task: [helm:build] helm dependency build charts/apache-jena
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "bhuwanupadhyay" chart repository
...Successfully got an update from the "ingress-nginx" chart repository
...Successfully got an update from the "haproxytech" chart repository
...Successfully got an update from the "traefik" chart repository
...Successfully got an update from the "neo4j" chart repository
...Successfully got an update from the "trucharts-library" chart repository
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 1 charts
Deleting outdated charts


In [40]:
!task helm:install CHART=apache-jena

task: [helm:install] helm upgrade --install apache-jena charts/apache-jena --namespace apache-jena --create-namespace --wait --timeout 10m
Release "apache-jena" does not exist. Installing it now.
NAME: apache-jena
LAST DEPLOYED: Fri Jun 16 16:00:12 2023
NAMESPACE: apache-jena
STATUS: deployed
REVISION: 1
NOTES:
1. Get the application URL by running these commands:
  http://jena.k8s.localdev/


In [39]:
!task helm:delete CHART=apache-jena

task: [helm:delete] helm delete apache-jena --namespace apache-jena
release "apache-jena" uninstalled


In [37]:
!pip install SPARQLWrapper

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
sparql_endpoint = "https://jena.k8s.localdev/ds"

sparql = SPARQLWrapper(sparql_endpoint)

In [3]:
def insert_data():
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        INSERT DATA {
            ex:subject1 rdf:type ex:Person ;
                         ex:name "John" .

            ex:subject2 rdf:type ex:Person ;
                         ex:name "Jane" .
        }
    """

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data inserted successfully.")

In [4]:
def search_data(sparql_query=None):
    if sparql_query is None:

        sparql_query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX ex: <http://example.org/>

            SELECT ?name
            WHERE {
                ?subject rdf:type ex:Person ;
                         ex:name ?name .
            }
        """

    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()

    if 'results' in results:
        bindings = results['results']['bindings']
        if len(bindings) > 0:
            print("Results:")
            for binding in bindings:
                name = binding['name']['value']
                print("Name:", name)
        else:
            print("No results found.")
    else:
        print("No results found.")

In [5]:
def search_data_by_name(name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        SELECT ?name
        WHERE {
            ?subject rdf:type ex:Person ;
                     ex:name ?name .
            FILTER (str(?name) = "%s")
        }
    """ % name
    search_data(sparql_query)

In [6]:
def update_data(subject_uri, new_name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        DELETE {{
            <{subject_uri}> ex:name ?old_name .
        }}
        INSERT {{
            <{subject_uri}> ex:name "{new_name}" .
        }}
        WHERE {{
            <{subject_uri}> rdf:type ex:Person ;
                            ex:name ?old_name .
        }}
    """.format(subject_uri=subject_uri, new_name=new_name)

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data updated successfully.")

In [7]:
insert_data()

Data inserted successfully.


In [8]:
search_data()

Results:
Name: John
Name: Jane


In [9]:
search_data_by_name("John")

Results:
Name: John


In [10]:
update_data("http://example.org/subject1", "John Doe")

Data updated successfully.


In [11]:
search_data()

Results:
Name: John Doe
Name: Jane


In [12]:
search_data_by_name("John Doe")

Results:
Name: John Doe
